In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
model = ChatOpenAI()

In [15]:
# 1st of all - define the state
class BlogState(TypedDict):
    title: str
    outline: str
    content: str
    evaluate: str

In [ ]:
# create a node to generating outlines

def create_outline(state: BlogState) -> BlogState:
    # fetch title
    title = state['title']

    # call llm and generate outline
    prompt = f"generate a detailed outline for a blog on the topic of - {title}"
    outline = model.invoke(prompt).content

    # update the state
    state['outline'] = outline
    return state

In [ ]:
 # create a node to generating blog
 
 def create_blog(state: BlogState) -> BlogState:

    #fetch title and outline
    title = state['title']
    outline = state['outline']

    # call the llm again and generate a blog
    prompt = f"write a detailed blog on the title - {title} using the following outline \n {outline}"
    content = model.invoke(prompt).content
    state['content'] = content

    return state

In [16]:
# create a node to evaluating blog

def evaluate_blog(state: BlogState) -> BlogState:

    #fetch title, outline, content
    title = state['title']
    outline = state['outline']
    content = state['content']

    # call the llm again and rate the blog based on outline
    prompt = f"Rate my blog based on this {outline} and generate an integer score from 1 to 10"
    evaluate = model.invoke(prompt).content
    state['evaluate'] = evaluate

    return state

In [17]:
# create the graph
graph = StateGraph(BlogState)

# add the nodes
graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog', evaluate_blog)

# add the edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', END)

workflow = graph.compile()

In [18]:
initial_state = {'title': 'Emerging of the AI in the Pakistan in 2025'}

final_state = workflow.invoke(initial_state)

print(final_state['title'])
print(final_state['outline'])
print(final_state['content'])

print(final_state['evaluate'])

Emerging of the AI in the Pakistan in 2025
I. Introduction
    A. Brief overview of Artificial Intelligence (AI) 
    B. Definition of AI
    C. Importance of AI in various sectors
    D. Introduction to the emergence of AI in Pakistan in 2025

II. Current scenario of AI in Pakistan
    A. Overview of AI technology in Pakistan
    B. AI adoption in different sectors (e.g. healthcare, finance, education)
    C. Challenges faced by Pakistan in adopting AI technology

III. Factors contributing to the emergence of AI in Pakistan in 2025
    A. Government initiatives and policies
    B. Investments in AI research and development
    C. Collaboration with international AI experts
    D. Increased awareness and acceptance of AI technology

IV. Impact of AI on different sectors in Pakistan
    A. Healthcare
        1. AI in disease diagnosis and treatment
        2. Telemedicine and remote healthcare services
    B. Finance
        1. AI in banking and financial services
        2. Fraud detec